In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env

In [ ]:
import pandas as pd

df = pd.read_csv('customer_shopping_behavior.csv', sep = ';')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

In [ ]:
df.isnull().sum()

We can see there are 37 missing values in review rating. To fill the missing values we will introduce median of review rating within each category.


In [ ]:
df['Review Rating'] = df['Review Rating'].fillna(df.groupby('Category')['Review Rating'].transform('median'))


In [ ]:
df.isnull().sum()

In [ ]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df = df.rename(columns = {'purchase_amount_(usd)' : 'purchase_amount'})

In [ ]:
df.columns

# Feature Engineering
## create a column age_group


In [ ]:
bins = [18, 30, 45, 60, 71]
labels = ['Young Adult', 'Adult', 'Middle-aged', 'Senior']

df['age_group'] = pd.cut(
    df['age'],
    bins=bins,
    labels=labels,
    right=False
)


In [ ]:
df[['age', 'age_group']].head(10)

# Create column purchase_frequency_days
It shows the number of days of purchases which helps to analyze the datset easily.

In [ ]:

frequency_mapping = {
    'Fortnightly': 14,
    'Bi-Weekly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Every 3 Months': 90,
    'Annually': 365
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)


# Change float datatype to integer.

df['purchase_frequency_days'] = df['purchase_frequency_days'].astype('Int64')   



In [ ]:

df[['purchase_frequency_days', 'frequency_of_purchases']].head(15)

In [ ]:
df[['discount_applied', 'promo_code_used']].head(15)

We can see from promo code column and discount applied gives the simliar data. 
It is obvious that if we use promocode, we can automaticall know that discount is applied.
But we have to check, is the data of both column are same. If they are similar we can delete one column to reduce data redundancy


In [ ]:
(df['discount_applied'] == df['promo_code_used']).all()

Since, the data are same in both columns, we delete promo_code column

In [ ]:
df = df.drop('promo_code_used', axis = 1)



In [ ]:
df.columns

In [ ]:
pip install psycopg2-binary sqlalchemy

In [ ]:
from sqlalchemy import create_engine

# Step 1: Database connection details
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
port = os.getenv("PG_PORT")
database = os.getenv("PG_DATABASE")

engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

print(repr(port))

# Step 2: Load DataFrame into PostgreSQL
table_name = "customer"        # table name in PostgreSQL

df.to_sql(
    table_name,
    engine,
    if_exists="replace",        # replace table if it already exists
    index=False                 # do not include DataFrame index
)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")
